# Project: Machine Learning

**Instructions for Students:**

Please carefully follow these steps to complete and submit your project:

1. **Make a copy of the Project**: Please make a copy of this project either to your own Google Drive or download locally. Work on the copy of the project. The master project is **Read-Only**, meaning you can edit, but it will not be saved when you close the master project. To avoid total loss of your work, remember to make a copy.

2. **Completing the Project**: You are required to work on and complete all tasks in the provided project. Be disciplined and ensure that you thoroughly engage with each task.
   
3. **Creating a Google Drive Folder**: Each of you must create a new folder on your Google Drive. This will be the repository for all your completed project files, aiding you in keeping your work organized and accessible.
   
4. **Uploading Completed Project**: Upon completion of your project, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
   
5. **Sharing Folder Link**: You're required to share the link to your project Google Drive folder. This is crucial for the submission and evaluation of your project.
   
6. **Setting Permission to Public**: Please make sure your Google Drive folder is set to public. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth project evaluation process for you and the reviewers.

## Project Description

In this Machine Learning Project, you will create your own supervised Machine Learning (ML) model. We will use the full FIFA21 Dataset and we will identify players that are above average.

We will use the column "Overall" with a treshold of 75 to define players that are 'Valuable'. This will become our target output which we need for a supervised ML model. Because we use the "Overall" as our target output, you cannot use "Overall" in your features, this will be explained further below.

This project will provide a comprehensive overview of your abilities in machine learning, from understanding the problem, choosing the right model, training, and optimizing it.

## Grading Criteria

Your score will be awarded based on the following criteria:
* 100: The model has an accuracy of more than 80% and an F1 score of more than 85%. This model is excellent and demonstrates a strong understanding of the task.
* 90: The model has an accuracy of more than 75% and an F1 score of more than 80%. This model is very good, with some room for improvement.
* 80: The model has an accuracy of more than 70% and an F1 score between 70% and 80%. This model is fairly good but needs improvement in balancing precision and recall.
* 70: The model has an accuracy of more than 65% and an F1 score between 60% and 70%. This model is below average and needs significant improvement.
* 60 or below: The model has an accuracy of less than 65% or an F1 score of less than 60%, or the student did not submit the accuracy and F1 score. This model is poor and needs considerable improvement.

Rmember to make a copy of this notebook in your Google Drive and work in your own copy.

Happy modeling!

>Note: If you get the accuracy of 100% and F1 score of 100%, while it may earn you good grades, it's an indication of overfitting.

## Student Identity

In [74]:
# @title #### Student Identity
student_id = "REAGEFDF" # @param {type:"string"}
name = "Hasan Mubarok" # @param {type:"string"}
drive_link = "https://drive.google.com/drive/folders/1dJEt75YsmLB5IBg5hFIijrHHatp97mAR?usp=sharing"  # @param {type:"string"}

assignment_id = "00_ml_project"

# Import grader package
!pip3 install rggrader scikit-learn
from rggrader import submit, submit_image

In [75]:
# Write any package/module installation that you need
# pip install goes here, this helps declutter your output below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

## Load the dataset and clean it

In this task, you will prepare and load your dataset. You need to download the full FIFA 21 Dataset from the link here: [Kaggle FIFA Player Stats Database](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?resource=download&select=FIFA21_official_data.csv).

>Note: Make sure you download FIFA 21 dataset.
>
>![FIFA21 Dataset](https://storage.googleapis.com/rg-ai-bootcamp/projects/fifa21_dataset-min.png)

After you download the dataset, you will then import the dataset then you will clean the data. For example there may be some empty cell in the dataset which you need to fill. Maybe there are also data that you need to convert to numeric value for analysis. Identify the data that is incomplete and fix them.

In the code block below, you can use the comments to guide you on what to do.

In [76]:
# Write your preprocessing and data cleaning here

# Load your data
file_path = "FIFA21_official_data.csv"
fifa_df = pd.read_csv(file_path, low_memory=False)
fifa_df

# Check your data for empty cell
fifa_df.isnull().sum()

# Fill the empty cell with data or drop the column
# categorical columns: 'Club', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'
# Drop columns Real Face, and Jersey Number as it is not important, drop Loaned From as it has too many missing values
fifa_df.drop(columns=['Real Face', 'Jersey Number', 'Loaned From'], inplace=True)
# Fill the empty cell for other columns with 'Unknown'
fifa_df[['Club', 'Body Type', 'Position']] = fifa_df[['Club', 'Body Type', 'Position']].fillna('Unknown')

# numerical columns: 'Volleys', 'Curve', 'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning', 'Vision', 'Composure', 'Marking', 'SlidingTackle', 'DefensiveAwareness'
columns = ['Volleys', 'Curve', 'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning', 'Vision', 'Composure', 'Marking', 'SlidingTackle', 'DefensiveAwareness']
# Fill the empty cell with the mean of the column
fifa_df[columns] = fifa_df[columns].fillna(fifa_df[columns].mean())

#financial columns: 'Value', 'Wage', 'Release Clause'
# Fill the empty cell with '0'
fifa_df[['Value', 'Wage', 'Release Clause']] = fifa_df[['Value', 'Wage', 'Release Clause']].fillna('0')


fifa_df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,176580,L. Suárez,33,https://cdn.sofifa.com/players/176/580/20_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,87,87,Atlético Madrid,https://cdn.sofifa.com/teams/240/light_30.png,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,87.0,€64.6M,57.0
1,192985,K. De Bruyne,29,https://cdn.sofifa.com/players/192/985/20_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/light_30.png,...,53.0,15.0,13.0,5.0,10.0,13.0,CAM,91.0,€161M,68.0
2,212198,Bruno Fernandes,25,https://cdn.sofifa.com/players/212/198/20_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,87,90,Manchester United,https://cdn.sofifa.com/teams/11/light_30.png,...,55.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,€124.4M,72.0
3,194765,A. Griezmann,29,https://cdn.sofifa.com/players/194/765/20_60.png,France,https://cdn.sofifa.com/flags/fr.png,87,87,FC Barcelona,https://cdn.sofifa.com/teams/241/light_30.png,...,49.0,14.0,8.0,14.0,13.0,14.0,ST,87.0,€103.5M,59.0
4,224334,M. Acuña,28,https://cdn.sofifa.com/players/224/334/20_60.png,Argentina,https://cdn.sofifa.com/flags/ar.png,83,83,Sevilla FC,https://cdn.sofifa.com/teams/481/light_30.png,...,79.0,8.0,14.0,13.0,13.0,14.0,LB,83.0,€46.2M,79.0


In [77]:
# drop the original 'Joined' and 'Contract Valid Until' columns
fifa_df = fifa_df.drop(['Joined', 'Contract Valid Until'], axis=1)

In [78]:
# Convert 'Height' to centimeters
fifa_df['Height'] = fifa_df['Height'].apply(lambda x: int(x.split("'")[0])*30.48 + int(x.split("'")[1])*2.54 if isinstance(x, str) else x)
# Convert 'Weight' to float
fifa_df['Weight'] = fifa_df['Weight'].str.rstrip('lbs').astype(float)

In [79]:
# handle financial columns
def convert_currency_to_float(df, columns):
    for column in columns:
        df[column] = df[column].replace(r'\€', '', regex=True)
        df[column] = df[column].apply(lambda x: float(x.replace('M', '')) * 1000000 if 'M' in x else (float(x.replace('K', '')) * 1000 if 'K' in x else float(x)))
    return df

columns = ['Value', 'Wage', 'Release Clause']
fifa_df = convert_currency_to_float(fifa_df, columns)
fifa_df[['Value', 'Wage', 'Release Clause']]

,Value,Wage,Release Clause
0,31500000.0,115000.0,64600000.0
1,87000000.0,370000.0,161000000.0
2,63000000.0,195000.0,124400000.0
3,50500000.0,290000.0,103500000.0
4,22000000.0,41000.0,46200000.0
...,...,...,...
17103,50000.0,500.0,120000.0
17104,50000.0,500.0,83000.0
17105,60000.0,500.0,149000.0
17106,50000.0,500.0,94000.0


## Build and Train your model

In this task you will analyze the data and select the features that is best at predicting if the Player is a 'Valuable' player or not.

The first step is to **define the target output** that you will use for training. Here's an example of how to create a target output:
- `df['OK Player'] = df['Overall'].apply(lambda x: 1 if x >= 50 else 0) #Define the OK Player using treshold of 50.`

Next you will **identify the features** that will best predict a 'Valuable' player. You are required to **submit the features you selected** in the Submission section below. Because we use the "Overall" as our target output, the use of "Overall" in your features is not allowed. You will automatically get 0 if you submit "Overall" in your features.

Once you identify the features, you will then **split the data** into Training set and Testing/Validation set.

Depending on the features you selected, **you may need to scale the features**.

Now you will **train your model, choose the algorithm** you are going to use carefully to make sure it gives the best result.

Once you have trained your model, you need to test the model effectiveness. **Make predictions against your Testing/Validation set** and evaluate your model. You are required to **submit the Accuracy Score and F1 score** in the Submission section below.

In the code block below, you can use the comments to guide you on what to do.

We have also provided 3 variables that you must use in your code, `ml_features`, `ml_accuracy` and `ml_f1_score`. You can move the variables around your code, assign values to them, but you cannot delete them.

In [80]:
# Write your code here

# Define the target output (Good >= 75)
fifa_df['Good'] = fifa_df['Overall'] >= 75

# Identify the features you will use in your model
ml_features = ['Age', 'Potential', 'Value', 'Wage', 'Special', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Body Type', 'Position', 'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'Release Clause']

# Split data into training set and test set
X = fifa_df[ml_features]
y = fifa_df['Good']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Scale the features (if needed, optional)
# Identify categorical features
categorical_features = ['Body Type', 'Position']

# Create a ColumnTransformer to handle categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [col for col in ml_features if col not in categorical_features]),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Fit and transform the training data
X_train = preprocessor.fit_transform(X_train)

# Transform the test data
X_test = preprocessor.transform(X_test)
scaler = StandardScaler(with_mean=False, with_std=True)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions using the test set
y_pred = model.predict(X_test)


# Evaluate the model
ml_accuracy = model.score(X_test, y_test)
ml_f1_score = f1_score(y_test, y_pred)

ml_accuracy, ml_f1_score



(0.9756477693356711, 0.9097472924187726)

## Submission

Once you are satisfied with the performance of your model, then you run the code block below to submit your project.


In [81]:
# Submit Method

# Do not change the code below
question_id = "01_ml_project_features"
submit(student_id, name, assignment_id, str(ml_features), question_id, drive_link)
question_id = "02_ml_project_accuracy"
submit(student_id, name, assignment_id, str(ml_accuracy), question_id, drive_link)
question_id = "03_ml_project_f1score"
submit(student_id, name, assignment_id, str(ml_f1_score), question_id, drive_link)

'Assignment successfully submitted'

## FIN